### Output from particular solution
The following code estimates the model under a given solution -- that is under a particular calibration and a given price level. Below is the baseline calibration used -- for any other calibrations, the price level should first be estimated in the Notebook "Full_estimation", and then updated below.

In [1]:
# Packages
from IPython.display import clear_output

from types import SimpleNamespace
import numpy as np
np.set_printoptions(suppress=True) # supresses scientific notation in numpy-arrays
from scipy import optimize
from scipy import interpolate
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
import warnings
from datetime import datetime
import pandas as pd

import one_run_model as model
%load_ext autoreload
%autoreload 2

### UNIT OF ACCOUNT IN THE MODEL IS MIO DKK
d = 1000000 # Converter from DKK to mDKK

In [2]:
warnings.filterwarnings("ignore") # See note below

starttime = datetime.now()
w0_vec      = np.load("w0_vec.npy",allow_pickle=True)
type_vec    = np.load("type_vec.npy",allow_pickle=True)

# PARAMETERS
par = SimpleNamespace()
par.rent = 1200*10/d # rent per m2 per year
par.uc_value = 0.01*0.8*10 # user cost per house value per year
par.uc_sqm = 300*10/d # user cost per m2 per year
par.r = ((1+0.015)**(10)-1)*0.67 # interest rate for loans, over 10 years, with 30% effective interest rate tax deduction
par.rho = (1+0.005)**(10)-1 # interest for deposits, over 10 years
par.LTI = 4/10/0.6 # 4 x yearly pre-tax income (10 years per period, tax of 40%)
par.LTV = .95 # 5% down payment rate
par.phi = 200000/d # Limit of uncollaterized loan

# HOUSEHOLDS (currently, only difference between them is wage path!)
# Baseline/average agent
hh_baseline = SimpleNamespace()
hh_baseline.wage_path = [1.5,2.5,3.5,2.5,1.5]
hh_baseline.theta = [1.30,1.30,1.30,1.30,0.9] # Utility boost from owning
hh_baseline.alpha = 0.7 # Share of consumption in contemporanous utility 
hh_baseline.eta = 1.5 # CRRA utility function parameter
hh_baseline.chi = 0.2 # Strength of bequest motive
hh_baseline.beta = 0.98**10 # discounting over 10 years
hh_baseline.n = 3000 # Number of this type of agent in each generation in simulation

# Low-ish constant wage (sygeplejerske/pædagog)
hh_low = SimpleNamespace()
hh_low.wage_path = [1.5,2,2,2,2]
hh_low.theta = [1.15,1.15,1.15,1.15,0.9] # Utility boost from owning
hh_low.alpha = 0.7 # Share of consumption in contemporanous utility 
hh_low.eta = 1.5 # CRRA utility function parameter
hh_low.chi = 0.2 # Strength of bequest motive
hh_low.beta = 0.98**10 # discounting over 10 years
hh_low.n = 1000 # Number of this type of agent in each generation in simulation

# # High-earner (kandidatgrad)
hh_high = SimpleNamespace()
hh_high.wage_path = [1.5,4,4,4,4]
hh_high.theta = [1.15,1.15,1.15,1.15,0.9] # Utility boost from owning
hh_high.alpha = 0.7 # Share of consumption in contemporanous utility 
hh_high.eta = 1.5 # CRRA utility function parameter
hh_high.chi = 0.2 # Strength of bequest motive
hh_high.beta = 0.98**10 # discounting over 10 years
hh_high.n = 1000 # Number of this type of agent in each generation in simulation

# SET ENDOGENOUS PARAMETER
par.p = 48000/d # owner price per sqm

df = pd.DataFrame()
df['type'] = type_vec
df['w0'] = w0_vec
df['u'] = None; df['c'] = None; df['h'] = None;df['s'] = None;df['wplus'] = None;df['buy'] = None;df['LTV'] = None;df['LTI'] = None

solution_baseline = model.estimate(par,hh_baseline)
solution_low = model.estimate(par,hh_low)
solution_high = model.estimate(par,hh_high)

for i in df.index:
    
    w0 = df.iloc[i]['w0']
    
    if df.loc[i,'type'] == 'baseline': solution = solution_baseline
    if df.loc[i,'type'] == 'low': solution = solution_low
    if df.loc[i,'type'] == 'high': solution = solution_high
        
    paths = model.create_paths(w0,*solution)

    df.at[i, 'u'] = paths.u_path
    df.at[i, 'c'] = paths.c_path
    df.at[i, 'h'] = paths.h_path
    df.at[i, 's'] = paths.s_path
    df.at[i, 'wplus'] = paths.wealth_path
    df.at[i, 'buy'] = paths.own_path
    df.at[i, 'LTV'] = paths.LTV_path
    df.at[i, 'LTI'] = paths.LTI_path
    clear_output(wait=True)
    print("Running for",datetime.now()-starttime)

###################################################
# Note - the above will throw a few warnings. 
# To the best of my knowledge, it is a non-issue,
# as it happens in the optimization process, and
# the optimizer simply don't choose those solutions
# (And in case 1D optimization fails, there's a
# 2D optimization fall back, which is defaulted to)
###################################################

Running for 0:00:25.138926


In [3]:
df = df.explode(['u','c','h','s','wplus','buy','LTV','LTI',])
df['demand'] = df['h']*df['buy']
df = df.fillna(0)

In [4]:
df.describe()

,w0,u,c,h,s,wplus,buy,LTV,LTI,demand
count,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.0,25000.000000,25000.000000
mean,0.096190,2.593027,1.681167,54.850082,-1.092716,0.503080,0.692160,0.0,0.687760,40.033992
std,0.491702,0.977189,0.490071,16.279526,1.059191,0.685994,0.461609,0.0,0.463416,30.381462
min,-1.833283,0.000000,0.000000,0.000000,-2.666667,-1.833283,0.000000,0.0,0.000000,0.000000
25%,-0.241329,1.982925,1.352238,40.690238,-1.666667,-0.124289,0.000000,0.0,0.000000,0.000000
50%,0.093012,2.765555,1.599441,55.478189,-1.333333,0.332726,1.000000,0.0,1.000000,41.374649
75%,0.425414,3.348377,1.724986,63.711300,-0.200000,1.241242,1.000000,0.0,1.000000,63.711300
max,1.857155,4.458804,2.715395,96.471710,1.049818,2.008024,1.000000,0.0,1.000000,96.471710
